In [1]:
import os
import numpy as np
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from PIL import Image

In [2]:
data_path = r'D:\studia mgr EIM\ZZSN\thecarconnectionpicturedataset'
listOfFiles = os.listdir(data_path)

In [ ]:
# opcja 1 - rozdzielenie na typy, w car_types przechowywane nazwy plikow

car_types= ['Van','Pickup','Station Wagon','4dr','3dr','2dr','SUV','Convertible']
# reszta to 'nan'
car_dictionary = dict.fromkeys(car_types)
for type_of_car in car_types:
    str_match = [s for s in listOfFiles if type_of_car in s]
    car_dictionary[type_of_car] = str_match

In [ ]:
## inicjalizacja parametrow
# rozmiar zdjec 320x213 pikseli, niektore na x240 piskeli, 
# TODO: trzeba funkcje, by były wszystkie w tym samym rozmiarze -> resize()

# image = Image.open('demo_image.jpg')
# new_image = image.resize((400, 400))
# new_image.save('image_400.jpg')  

# z czego korzystam? https://jovian.ai/tvscitechtalk/car-gan
# https://github.com/ozanciga/gans-with-pytorch/blob/master/cgan/cgan.py
# https://github.com/eriklindernoren/PyTorch-GAN/blob/36d3c77e5ff20ebe0aeefd322326a134a279b93e/implementations/cgan/cgan.py#L189


# stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
# train_ds = ImageFolder(DATA_DIR, transform=T.Compose([
#     T.Resize(image_size),
#     T.CenterCrop(image_size),
#     T.ToTensor(),
#     T.Normalize(*stats)])) # normalizacja, zeby dyskryminator mial latwiej-?

# train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)


In [ ]:
## klasa generatora
# nn.Module - base class for all neural network modules
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

In [ ]:
## klasa dyskryminatora
class Discriminator(nn.Module):
    def __init__(self):
            super(Discriminator, self).__init__()

In [ ]:
# entropia krzyzowa 
# TODO: ustalenie parametrow

# torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=- 100, 
#     reduce=None, reduction='mean', label_smoothing=0.0)